In [ ]:
import sqlite3
import pandas as pd
import os

# 1. Connect to the Database
db_path = os.path.join('..', 'database', 'agri_project.db')
conn = sqlite3.connect(db_path)

print("✅ Connection Established.")

# 2. Check Raw Data (Bronze Layer)
df_raw = pd.read_sql_query("SELECT * FROM yield_data LIMIT 5;", conn)
display(df_raw)

In [5]:
# --- STEP 2: CREATE DIMENSION TABLES ---
# We use 'executescript' to run multiple SQL commands at once.

dim_creation_script = """
-- 1. Dimension: Crops (The 'What')
DROP TABLE IF EXISTS dim_crops;

CREATE TABLE dim_crops AS
SELECT DISTINCT item as crop_name
FROM yield_data
ORDER BY crop_name;

-- 2. Dimension: Countries (The 'Where')
DROP TABLE IF EXISTS dim_countries;

CREATE TABLE dim_countries AS
SELECT DISTINCT country as country_name
FROM yield_data
ORDER BY country_name;
"""

conn.executescript(dim_creation_script)
print("✅ Dimension tables created: 'dim_crops' and 'dim_countries'.")

✅ Dimension tables created: 'dim_crops' and 'dim_countries'.


In [6]:
# --- STEP 3: CREATE FACT TABLE ---

fact_creation_script = """
DROP TABLE IF EXISTS fact_crop_yields;

CREATE TABLE fact_crop_yields AS
SELECT 
    -- Foreign Keys (linking to dimensions)
    c.rowid as country_id,
    cr.rowid as crop_id,
    
    -- Time
    y.year,
    
    -- Metrics (The Facts)
    y.yield as yield_hg_ha,
    y.average_rain_fall_mm_per_year as rain_mm,
    y.avg_temp,
    y.pesticides_tonnes as pesticides_tonnes
    
FROM yield_data y
JOIN dim_countries c ON y.country = c.country_name
JOIN dim_crops cr ON y.item = cr.crop_name;
"""

conn.executescript(fact_creation_script)
print("✅ Fact table updated with Pesticides column.")

✅ Fact table updated with Pesticides column.


In [11]:
# --- STEP 4: ADVANCED ANALYSIS (Window Functions) ---

# We want to analyze: Did productivity grow or shrink compared to the previous year?
# We will focus on one crop to make the data clear.

query_yoy = """
WITH AnnualYield AS (
    -- 1. Get the raw data for a specific crop (e.g., Potatoes)
    SELECT 
        f.year,
        AVG(f.yield_hg_ha) as current_yield
    FROM fact_crop_yields f
    JOIN dim_crops c ON f.crop_id = c.rowid
    WHERE c.crop_name = 'Maize'
    GROUP BY f.year
)
SELECT 
    year,
    current_yield,
    -- 2. LAG: Look at the PREVIOUS row's yield
    LAG(current_yield) OVER (ORDER BY year) as previous_year_yield,
    
    -- 3. Calculate Growth %: ((Current - Previous) / Previous) * 100
    ROUND(
        (current_yield - LAG(current_yield) OVER (ORDER BY year)) / 
        LAG(current_yield) OVER (ORDER BY year) * 100
    , 2) as growth_pct
FROM AnnualYield;
"""

df_growth = pd.read_sql_query(query_yoy, conn)

# Let's see the result. 
# The first year will be NaN (Not a Number) because there is no "previous year" for 1990.
display(df_growth.head(10))

,year,current_yield,previous_year_yield,growth_pct
0,1990,27706.041916,NaN,NaN
1,1991,27900.958084,27706.041916,0.70
2,1992,27962.977528,27900.958084,0.22
3,1993,28997.363128,27962.977528,3.70
4,1994,28344.642458,28997.363128,-2.25
5,1995,29200.106145,28344.642458,3.02
6,1996,30763.502793,29200.106145,5.35
7,1997,31915.184358,30763.502793,3.74
8,1998,32526.022346,31915.184358,1.91
9,1999,33003.357542,32526.022346,1.47


In [10]:
# --- STEP 6: CREATING VIEWS ---
# We will create a view that automatically flags outliers.

view_script = """
DROP VIEW IF EXISTS view_data_anomalies;

CREATE VIEW view_data_anomalies AS
SELECT 
    c.country_name,
    cr.crop_name,
    f.year,
    f.avg_temp,
    f.yield_hg_ha
FROM fact_crop_yields f
JOIN dim_countries c ON f.country_id = c.rowid
JOIN dim_crops cr ON f.crop_id = c.rowid
WHERE 
    f.avg_temp > 40  -- Suspiciously hot
    OR f.yield_hg_ha = 0; -- Zero yield (Crop failure or Error)
"""

conn.executescript(view_script)
print("✅ View 'view_data_anomalies' created successfully.")

# Let's test the view
df_anomalies = pd.read_sql_query("SELECT * FROM view_data_anomalies", conn)

if df_anomalies.empty:
    print("🎉 Good news! No anomalies found in the current dataset.")
else:
    print("⚠️ Anomalies detected:")
    display(df_anomalies)

✅ View 'view_data_anomalies' created successfully.
🎉 Good news! No anomalies found in the current dataset.
